<h1>2do Parcial</h1>
<h2>Nombre: Fernandez Acho Alexander Paul
Carrera: Ing. de Sistemas</h2>

<h3>Desarrolle un cuadernillo aplicando aprendizaje por refuerzo, para que un agente pueda jugar cuatro en raya.

</h3>


In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd


class Connect4Board:
    def __init__(self, rows=6, cols=7):
        self.rows = rows
        self.cols = cols
        self.state = np.zeros((rows, cols), dtype=int)
    
    def valid_moves(self):
        """Devuelve las columnas donde se puede colocar una ficha"""
        return [col for col in range(self.cols) if self.state[0, col] == 0]
    
    def update(self, symbol, col):
        """Coloca una ficha en la columna especificada"""
        if col not in self.valid_moves():
            raise ValueError(f"Movimiento ilegal en columna {col}!")
        
        # Encontrar la fila más baja disponible en esa columna
        for row in range(self.rows - 1, -1, -1):
            if self.state[row, col] == 0:
                self.state[row, col] = symbol
                return row, col
        
        raise ValueError("Columna llena!")
    
    def check_win(self, row, col, symbol):
        """Verifica si hay 4 en línea desde la posición dada"""
        directions = [
            (0, 1),   # horizontal
            (1, 0),   # vertical
            (1, 1),   # diagonal /
            (1, -1)   # diagonal \
        ]
        
        for dr, dc in directions:
            count = 1  # cuenta la ficha actual
            
            # Contar en una dirección
            r, c = row + dr, col + dc
            while 0 <= r < self.rows and 0 <= c < self.cols and self.state[r, c] == symbol:
                count += 1
                r, c = r + dr, c + dc
            
            # Contar en la dirección opuesta
            r, c = row - dr, col - dc
            while 0 <= r < self.rows and 0 <= c < self.cols and self.state[r, c] == symbol:
                count += 1
                r, c = r - dr, c - dc
            
            if count >= 4:
                return True
        
        return False
    
    def is_game_over(self):
        """
        Retorna:
        - 1 si gana el jugador 1
        - -1 si gana el jugador -1
        - 0 si es empate
        - None si el juego continúa
        """
        # Verificar si hay ganador
        for row in range(self.rows):
            for col in range(self.cols):
                if self.state[row, col] != 0:
                    if self.check_win(row, col, self.state[row, col]):
                        return self.state[row, col]
        
        # Verificar empate (tablero lleno)
        if len(self.valid_moves()) == 0:
            return 0
        
        # El juego continúa
        return None
    
    def reset(self):
        """Reinicia el tablero"""
        self.state = np.zeros((self.rows, self.cols), dtype=int)
    
    def display(self):
        """Muestra el tablero de forma visual"""
        print("\n  " + " ".join([str(i) for i in range(self.cols)]))
        print("  " + "-" * (self.cols * 2 - 1))
        for row in self.state:
            print("| " + " ".join([str(int(cell)) if cell != 0 else "." for cell in row]) + " |")
        print("  " + "-" * (self.cols * 2 - 1))


In [2]:
class Connect4Game:
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Connect4Board()
    
    def selfplay(self, rounds=100):
        """Ejecuta múltiples partidas de auto-juego"""
        wins = [0, 0]
        draws = 0
        
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            
            game_over = False
            last_move = None
            
            while not game_over:
                for player_idx, player in enumerate(self.players):
                    try:
                        col = player.move(self.board)
                        row, col = self.board.update(player.symbol, col)
                        last_move = (row, col, player.symbol)
                        
                        # Actualizar el estado para ambos jugadores
                        for p in self.players:
                            p.update(self.board)
                        
                        # Verificar si el juego terminó
                        result = self.board.is_game_over()
                        if result is not None:
                            game_over = True
                            break
                            
                    except ValueError:
                        # Movimiento ilegal, el jugador pierde automáticamente
                        result = -player.symbol
                        game_over = True
                        break
            
            # Dar recompensas
            self.reward()
            
            # Contar victorias
            result = self.board.is_game_over()
            if result == 1:
                wins[0] += 1
            elif result == -1:
                wins[1] += 1
            else:
                draws += 1
        
        return wins, draws
    
    def reward(self):
        """Asigna recompensas a los jugadores según el resultado"""
        winner = self.board.is_game_over()
        
        if winner == 0:  # Empate
            for player in self.players:
                player.reward(0.5)
        else:  # Hay ganador
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1.0)  # Victoria
                else:
                    player.reward(0.0)  # Derrota
    
    def play_human_vs_agent(self, human_first=True):
        """Permite jugar contra el agente entrenado"""
        self.board.reset()
        human_symbol = 1 if human_first else -1
        agent = self.players[0] if not human_first else self.players[1]
        
        current_player = 1 if human_first else -1
        
        print("¡Bienvenido al Cuatro en Raya!")
        print(f"Eres el jugador {human_symbol}")
        self.board.display()
        
        while True:
            if current_player == human_symbol:
                # Turno humano
                try:
                    col = int(input(f"Tu turno (columnas 0-{self.board.cols-1}): "))
                    if col not in self.board.valid_moves():
                        print("Movimiento inválido. Intenta de nuevo.")
                        continue
                    self.board.update(current_player, col)
                except (ValueError, IndexError):
                    print("Entrada inválida. Usa números del 0 al 6.")
                    continue
            else:
                # Turno del agente
                col = agent.move(self.board, explore=False)  # Sin exploración en juego real
                print(f"El agente juega en la columna {col}")
                self.board.update(current_player, col)
            
            self.board.display()
            
            # Verificar fin del juego
            result = self.board.is_game_over()
            if result is not None:
                if result == human_symbol:
                    print("¡Felicitaciones! ¡Has ganado!")
                elif result == -human_symbol:
                    print("El agente ha ganado. ¡Mejor suerte la próxima vez!")
                else:
                    print("¡Empate!")
                break
            
            # Cambiar turno
            current_player *= -1

In [3]:
class Connect4Agent:
    def __init__(self, alpha=0.1, prob_exp=0.3, decay_rate=0.995):
        self.value_function = {}  # Tabla estado -> valor
        self.alpha = alpha        # Tasa de aprendizaje
        self.prob_exp = prob_exp  # Probabilidad de exploración inicial
        self.decay_rate = decay_rate  # Decaimiento de exploración
        self.positions = []       # Estados visitados en la partida actual
        self.symbol = None
    
    def reset(self):
        """Reinicia para una nueva partida"""
        self.positions = []
    
    def get_state_key(self, board_state):
        """Convierte el estado del tablero en una clave única"""
        return str(board_state.reshape(-1))
    
    def move(self, board, explore=True):
        """Selecciona la mejor jugada usando epsilon-greedy"""
        valid_moves = board.valid_moves()
        
        if not valid_moves:
            raise ValueError("No hay movimientos válidos")
        
        # Exploración (epsilon-greedy)
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            return np.random.choice(valid_moves)
        
        # Explotación: elegir la mejor jugada
        best_value = -float('inf')
        best_moves = []
        
        for col in valid_moves:
            # Simular el movimiento
            next_board = board.state.copy()
            for row in range(board.rows - 1, -1, -1):
                if next_board[row, col] == 0:
                    next_board[row, col] = self.symbol
                    break
            
            state_key = self.get_state_key(next_board)
            value = self.value_function.get(state_key, 0.0)
            
            if value > best_value:
                best_value = value
                best_moves = [col]
            elif value == best_value:
                best_moves.append(col)
        
        # Si hay empate, elegir aleatoriamente entre las mejores
        return np.random.choice(best_moves)
    
    def update(self, board):
        """Registra el estado actual"""
        state_key = self.get_state_key(board.state)
        self.positions.append(state_key)
    
    def reward(self, reward):
        """Actualiza la función de valor usando TD learning"""
        # Retropropagación temporal de la recompensa
        for i, state in enumerate(reversed(self.positions)):
            if state not in self.value_function:
                self.value_function[state] = 0.0
            
            # TD Update: V(s) = V(s) + α[R - V(s)]
            self.value_function[state] += self.alpha * (reward - self.value_function[state])
            
            # Decaimiento de la recompensa para estados anteriores
            reward = self.value_function[state] * 0.9
    
    def decay_exploration(self):
        """Reduce la probabilidad de exploración"""
        self.prob_exp *= self.decay_rate

In [ ]:
# Ejemplo de uso y entrenamiento
if __name__ == "__main__":
    # Crear agentes
    agent1 = Connect4Agent(alpha=0.1, prob_exp=0.3)
    agent2 = Connect4Agent(alpha=0.1, prob_exp=0.3)
    
    # Crear juego
    game = Connect4Game(agent1, agent2)
    
    print("Entrenando agentes...")
    
    # Entrenar en fases
    phases = [
        (50000, 0.3),   # Fase 1: Mucha exploración
        (30000, 0.2),   # Fase 2: Exploración media
        (20000, 0.1),   # Fase 3: Poca exploración
    ]
    
    total_wins_1, total_wins_2, total_draws = 0, 0, 0
    
    for rounds, exploration in phases:
        agent1.prob_exp = exploration
        agent2.prob_exp = exploration
        
        wins, draws = game.selfplay(rounds)
        total_wins_1 += wins[0]
        total_wins_2 += wins[1]
        total_draws += draws
        
        print(f"Fase completada - Exploración: {exploration}")
        print(f"Victorias Agent1: {wins[0]}, Agent2: {wins[1]}, Empates: {draws}")
    
    print(f"\nEntrenamiento completado!")
    print(f"Resultados totales:")
    print(f"Agent1: {total_wins_1} victorias")
    print(f"Agent2: {total_wins_2} victorias") 
    print(f"Empates: {total_draws}")
    
    # Mostrar estados más valiosos para el agente 1
    print(f"\nAgent1 aprendió {len(agent1.value_function)} estados diferentes")
    
    # Top 10 estados más valiosos
    top_states = sorted(agent1.value_function.items(), key=lambda x: x[1], reverse=True)[:10]
    print("\nTop 10 estados más valiosos para Agent1:")
    for i, (state, value) in enumerate(top_states, 1):
        print(f"{i}. Valor: {value:.4f}")
        
    

Entrenando agentes...


100%|██████████| 50000/50000 [16:55<00:00, 49.25it/s]


Fase completada - Exploración: 0.3
Victorias Agent1: 28080, Agent2: 21861, Empates: 59


100%|██████████| 30000/30000 [08:32<00:00, 58.55it/s]


Fase completada - Exploración: 0.2
Victorias Agent1: 17932, Agent2: 12044, Empates: 24


100%|██████████| 20000/20000 [05:18<00:00, 62.83it/s]


Fase completada - Exploración: 0.1
Victorias Agent1: 13233, Agent2: 6756, Empates: 11

Entrenamiento completado!
Resultados totales:
Agent1: 59245 victorias
Agent2: 40661 victorias
Empates: 94

Agent1 aprendió 434622 estados diferentes

Top 10 estados más valiosos para Agent1:
1. Valor: 1.0000
2. Valor: 1.0000
3. Valor: 1.0000
4. Valor: 1.0000
5. Valor: 1.0000
6. Valor: 1.0000
7. Valor: 1.0000
8. Valor: 1.0000
9. Valor: 1.0000
10. Valor: 1.0000


KeyboardInterrupt: Interrupted by user